In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x77c07f0


In [2]:
%jsroot on

In [3]:
# FIXME: don't touch! the fortran code has a global state :(
# ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
ROOT.gSystem.Load("OO/diehl/libObservable.so")

0

In [5]:
ROOT.gInterpreter.Declare("#include \"OO/diehl/OOTools.h\"")

True

In [6]:
# use same masses as in whizard!
mW = 80.419
mZ = 91.1882

ROOT.OOTools.init(250, mW, mZ)

In [7]:
ROOT.ildStyle.SetOptStat(1)
# ROOT.ildStyle.SetOptStat(0)

In [8]:
%%cpp
using namespace ROOT::VecOps;

In [9]:
n_points = 5000000

df = ROOT.RDataFrame(n_points)

In [10]:
df = df.Define("co", "gRandom->Uniform(-1., 1.)")
df = df.Define("co1", "gRandom->Uniform(-1., 1.)")
df = df.Define("co2", "gRandom->Uniform(-1., 1.)")
df = df.Define("ph1", "gRandom->Uniform(-1.*ROOT::Math::Pi(), 1.*ROOT::Math::Pi())")
df = df.Define("ph2", "gRandom->Uniform(-1.*ROOT::Math::Pi(), 1.*ROOT::Math::Pi())")

In [11]:
df = df.Define("proba", "OOTools::proba(co, co1, co2, ph1, ph2, 0, true, 0)")

In [12]:
pi = ROOT.Math.Pi()

# or just fill 1D histograms with weight proba??
# p_co = df.Profile1D(("", ";co", 25, -1., 1.), "co", "proba")
# p_co1 = df.Profile1D(("", ";co1", 25, -1., 1.), "co1", "proba")
# p_co2 = df.Profile1D(("", ";co2", 25, -1., 1.), "co2", "proba")
# p_ph1 = df.Profile1D(("", ";ph1", 25, -pi, pi), "ph1", "proba")
# p_ph2 = df.Profile1D(("", ";ph2", 25, -pi, pi), "ph2", "proba")
h_co  = df.Histo1D(("", ";co", 25, -1., 1.), "co", "proba")
h_co1 = df.Histo1D(("", ";co1", 25, -1., 1.), "co1", "proba")
h_co2 = df.Histo1D(("", ";co2", 25, -1., 1.), "co2", "proba")
h_ph1 = df.Histo1D(("", ";ph1", 25, -pi, pi), "ph1", "proba")
h_ph2 = df.Histo1D(("", ";ph2", 25, -pi, pi), "ph2", "proba")

In [ ]:
c_co = ROOT.TCanvas()
# p_co.SetMinimum(0)
# p_co.Draw()
h_co.SetMinimum(0)
h_co.Draw()
c_co.Draw()

c_co1 = ROOT.TCanvas()
# p_co1.SetMinimum(0)
# p_co1.Draw()
h_co1.SetMinimum(0)
h_co1.Draw()
c_co1.Draw()

c_ph1 = ROOT.TCanvas()
# p_ph1.SetMinimum(0)
# p_ph1.Draw()
h_ph1.SetMinimum(0)
h_ph1.Draw()
c_ph1.Draw()

c_co2 = ROOT.TCanvas()
# p_co2.SetMinimum(0)
# p_co2.Draw()
h_co2.SetMinimum(0)
h_co2.Draw()
c_co2.Draw()

c_ph2 = ROOT.TCanvas()
# p_ph2.SetMinimum(0)
# p_ph2.Draw()
h_ph2.SetMinimum(0)
h_ph2.Draw()
c_ph2.Draw()